In [2]:
import numpy as np

In [4]:
from sklearn import datasets

n_points = 5000
X, color = datasets.make_s_curve(n_points, random_state=0)
n_neighbors = 10
n_components = 2

In [5]:
import torch

In [6]:
data = torch.from_numpy(X)

In [7]:
N, D = data.shape
K = 10
d = 2
eps = 1e-4

In [8]:
dist = torch.cdist(data, data)
_, neighbors_idx = dist.topk(K+1, 1, largest=False)
neighbors_idx = neighbors_idx[:, 1:]
neighbors = data[neighbors_idx, :]  # N x K x D

In [9]:
G = data.unsqueeze(1).repeat(1, K, 1) - neighbors
G = torch.bmm(G, G.permute(0, 2, 1))
G = G + torch.eye(K).unsqueeze(0).repeat(N, 1, 1) * eps
weights = torch.linalg.solve(G, torch.ones(N, K, dtype=G.dtype))
weights = weights / weights.sum(dim=1, keepdims=True)

In [10]:
weights = torch.scatter(torch.zeros(N, N, dtype=weights.dtype), 1, neighbors_idx, weights)

In [11]:
L = torch.eye(N, dtype=weights.dtype) - weights
M = L.T.mm(L)
eigvalue, eigvector = torch.linalg.eigh(M)
embedding = eigvector[:, 1:1+d]